# Maternal mortality ratio compared to the female population with at least some secondary education (2005-2012)

## Setup

In [1]:
from __future__ import unicode_literals, print_function, division

In [2]:
import os

In [3]:
import numpy as np
import pandas as pd

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

In [4]:
import plot_constants as pc

## Specific Configuration

In [5]:
DATA_DIR = os.path.join('.', 'data')
RAW_FILE = os.path.join(DATA_DIR, 'raw_data_20150701.xlsx')

In [6]:
SHEET = 'Figure30'

PARSE_COLS = 'A:C'
SKIPROWS = 0
SKIP_FOOTER = 54-48+2

title = 'Risk of Maternal Mortality by Secondary Education'
subtitle = '(Selected countries, 2005-2012)'
source = dict(label='United Nations Development Programme (UNDP)', 
              link='http://hdr.undp.org/en/content/table-4-gender-inequality-index',
              accessed='NNN')

X_LABEL = 'Female population with at least some secondary education (% aged 25 and above), 2005-2012'
Y_LABEL = 'Maternal mortality ratio (deaths per 100,000 live births), 2010'

## Data Importation and Munging

In [7]:
DF = pd.read_excel(RAW_FILE,
                   sheetname=SHEET,
                   parse_cols=PARSE_COLS,
                   skiprows=SKIPROWS,
                   skip_footer=SKIP_FOOTER)
DF.columns = ['country', 'mortality', 'secondary']
DF = pc.clean_mkd(DF, 'country')
DF = pc.clean_mda(DF, 'country')
DF

,country,mortality,secondary
0,Austria,4,100.000000
1,Estonia,2,100.000000
2,Finland,5,100.000000
3,Luxembourg,20,100.000000
4,Czech Republic,5,99.865760
5,United Kingdom,12,99.817870
6,Kazakhstan,51,99.250040
7,Slovakia,6,99.094930
8,Latvia,34,98.912290
9,Hungary,21,97.900000


## Plotting

In [8]:
plot_height = pc.PLOT_HEIGHT
plot_width = pc.PLOT_WIDTH
filename = 'vaw/fig_15'

trace = Scatter(
    x=DF.secondary,
    y=DF.mortality,
    mode='markers+text',
    text=DF.country,
    textposition='top center',
    textfont=Font(
        family='Arial',
    ),
    marker=Marker(color=pc.FEMALE_FILL,
                  line=Line(color=pc.FEMALE_LINE,
                            width=1,
                           ),
                  symbol=pc.FEMALE_SYMBOL,
                  size=10,)
)

data = Data([trace])

layout = Layout(
    title = '<b>{}</b><br>{}'.format(title, subtitle),
    autosize=False,
    width=1000,
    height=plot_height,
    hovermode='closest',
    margin=Margin(
        l=120,
        r=120,
        pad=0
    ),
    xaxis=XAxis(title=X_LABEL),
    yaxis=YAxis(dict(zeroline=False), title=Y_LABEL),
    annotations=Annotations([
        Annotation(
            x=0.5,
            y=-0.1,
            xref='paper',
            yref='paper',
            text='Source: <a href="{}">{}</a>'.format(source['link'],source['label']),
            align='center',
            showarrow=False
        )
    ]),
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename=filename, height=plot_height)

## Alternative with Mean Years of Schooling

In [9]:
SHEET = 'Figure21'

PARSE_COLS = 'A:C'
SKIPROWS = 0
SKIP_FOOTER = 52-48

X_LABEL = 'Mean years of schooling (female, 2002-2012)'

DF_SCHOOL = pd.read_excel(RAW_FILE,
                   sheetname=SHEET,
                   parse_cols=PARSE_COLS,
                   skiprows=SKIPROWS,
                   skip_footer=SKIP_FOOTER)
DF_SCHOOL.columns = ['country', 'schooling', 'birthrate']
DF_SCHOOL = pc.clean_mkd(DF_SCHOOL, 'country')
DF_SCHOOL = pc.clean_mda(DF_SCHOOL, 'country')

DF_ALT = pd.merge(DF, DF_SCHOOL)[['country', 'mortality', 'schooling']]
DF_ALT.head()

,country,mortality,schooling
0,Austria,4,8.880000
1,Estonia,2,12.280000
2,Finland,5,10.330000
3,Luxembourg,20,10.922258
4,Czech Republic,5,12.140000


In [10]:
x1, x2 = DF_ALT.schooling.min(), DF_ALT.schooling.max()

m, c = np.polyfit(DF_ALT.schooling, DF_ALT.mortality, 1)
y1 = m * x1 + c
y2 = m * x2 + c

x1, y1, x2, y2

(6.380445371, 24.836633422054945, 12.82006784, 11.754466420339696)

In [11]:
plot_height = pc.PLOT_HEIGHT
plot_width = pc.PLOT_WIDTH
filename = 'vaw/fig_15_mean_school_vs_mortality'

dots = Scatter(
    x=DF_ALT.schooling,
    y=DF_ALT.mortality,
    mode='markers+text',
    text=DF_ALT.country,
    textposition='top center',
    textfont=Font(
        family='Arial',
    ),
    name='Correlation',
    marker=Marker(color=pc.FEMALE_FILL,
                  line=Line(color=pc.FEMALE_LINE,
                            width=1,
                           ),
                  symbol=pc.FEMALE_SYMBOL,
                  size=10,)
)

line_best_fit = Scatter(
    x=[x1, x2],
    y=[y1, y2],
    mode='lines',
    name='Fitted line',
)

data = Data([dots, line_best_fit])

layout = Layout(
    title = '<b>{}</b><br>{}'.format(title, subtitle),
    showlegend=False,
    autosize=False,
    width=1000,
    height=plot_height,
    hovermode='closest',
    margin=Margin(
        l=120,
        r=120,
        pad=0
    ),
    xaxis=XAxis(title=X_LABEL),
    yaxis=YAxis(dict(zeroline=False), title=Y_LABEL),
    annotations=Annotations([
        Annotation(
            x=0.5,
            y=-0.1,
            xref='paper',
            yref='paper',
            text='Source: <a href="{}">{}</a>'.format(source['link'],source['label']),
            align='center',
            showarrow=False
        )
    ]),
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename=filename, height=plot_height)

### Check label: TFYR Macedonia

In [12]:
from IPython.display import display_html
import plot
reload(plot)

rec = plot.PlotRecord(locals())
rec.make_pages()
print(rec)

DF: <class 'pandas.core.frame.DataFrame'>
X_LABEL: Mean years of schooling (female, 2002-2012)
Y_LABEL: Maternal mortality ratio (deaths per 100,000 live births), 2010
caption: <type 'NoneType'>
fig: <class 'plotly.graph_objs.graph_objs.Figure'>
fig_no: 15_mean_school_vs_mortality
filename: vaw/fig_15_mean_school_vs_mortality
footnotes: <type 'NoneType'>
plot_height: <type 'int'>
plot_width: <type 'int'>
record: <type 'dict'>
source_accessed: <type 'NoneType'>
source_label: United Nations Development Programme (UNDP)
source_link: http://hdr.undp.org/en/content/table-4-gender-inequality-index
subtitle: (Selected countries, 2005-2012)
thumb: <img src="https://api.plot.ly/v2/files/gauden:457/image?image_name=block-thumb" alt="Risk of Maternal Mortality by Secondary Education">
time_epoch: <type 'float'>
time_human: <type 'str'>
title: Risk of Maternal Mortality by Secondary Education
url_embed: https://plot.ly/~gauden/457.embed
url_svg: https://plot.ly/~gauden/457.svg

